# **Workflow for TYSAR StripMap Coregistrastion Stack [Configure]**
This workflow is designed to be taken over by MintPy Process.

-> **Minimum Requirements:**
- [install ISCE2](https://github.com/InSARProcessing/isce2/blob/develop/isce%E5%AE%89%E8%A3%85%E7%AC%94%E8%AE%B0.md)
- TYSAR StripMap data -> You need to unzip all of them
- Local SRTM GL1 DEM tiles

-> **Steps:**
- Mosaic local SRTM GL1 DEM tiles
- Prepare StripMap data
- Stack StripMap data
- Generate configs
- Run ISCE2 stack

In [ ]:
import os
import subprocess

# 1. Input ISCE repository location
isce2_install = "/home/wang_xuanxuan/tools/isce2"
os.environ["PATH"] = os.path.join(isce2_install, "install", "bin") + ":" + os.environ["PATH"]
# Check if the PYTHONPATH exists, if not, initialize it as an empty string
pythonpath = os.environ.get("PYTHONPATH", "")
os.environ["PYTHONPATH"] = os.path.join(isce2_install, "install", "packages") + ":" + pythonpath

# 2. Input DEM and SLC paths
DEM_PATH="/mnt/g/Shenzhen_ty/DEM"
SLC_PATH = "/mnt/g/Shenzhen_ty/SLC"
SLC_ISCE_PATH = "/mnt/g/Shenzhen_ty/SLC_ISCE"
ISCE_WORKSPACE="/home/wang_xuanxuan/Py_projects/output/SHENZHEN_Stack_2"

# 3. Input DEM region, T_MAX, B_MAX, REF_DATE, and ROI
DEM_REGION="22 24 113 115"
T_MAX="35"
B_MAX="120"
REF_DATE="20231207" # 参考日期，也将作为长时序标记日期
ROI="22.68 22.77 114.23 114.3"

<b> 1. Download DEM </b>

In [ ]:
os.chdir(DEM_PATH)
dem_cmd = f"dem.py -a stitch -b {DEM_REGION} -r -s 1 -c -d ."
subprocess.run(dem_cmd, shell=True, check=True)

<b> 2. Prepare SLC（Batch） </b>

In [ ]:
os.environ["PYTHONPATH"] = os.environ.get("PYTHONPATH", "") \
                           + os.pathsep \
                           + os.path.join(isce2_install, "src", "isce2", "contrib", "stack")
os.environ["PATH"] += os.pathsep + os.path.join(isce2_install, "src", "isce2", "contrib", "stack", "stripmapStack")

os.chdir(ISCE_WORKSPACE)
prepare_slc = f"prepareSpacety.py -i {SLC_PATH} -o {SLC_ISCE_PATH}"
subprocess.run(prepare_slc, shell=True, check=True)

<b> 3. Generate config files </b>

In [ ]:
import os
import glob
os.chdir(ISCE_WORKSPACE)

os.environ["PYTHONPATH"] = os.environ.get("PYTHONPATH", "") + os.pathsep + os.path.join(isce2_install, "src", "isce2", "contrib", "stack")
os.environ["PATH"] += os.pathsep + os.path.join(isce2_install, "src", "isce2", "contrib", "stack", "stripmapStack")

DEM_WGS84 = glob.glob(os.path.join(DEM_PATH, "*.wgs84"))[0]
#stackSM = f"""/home/wang_xuanxuan/tools/src/isce2/contrib/stack/stripmapStack/stackStripMap.py \
stackSM = f"""stackStripMap.py \
-s {SLC_ISCE_PATH} \
-d {DEM_WGS84} \
-t {T_MAX} -b {B_MAX} \
-W interferogram \
-u snaphu \
-useGPU \
-a 3 -r 3 \
--nofocus \
-m {REF_DATE} \
-x "{ROI}"
""" 
subprocess.run(stackSM, shell=True, check=True)

<b> 4. Run files </b>

In [ ]:
os.chdir(ISCE_WORKSPACE)
cmd_crop = "bash ./run_files/run_01_crop"
subprocess.run(cmd_crop, shell=True, check=True)

In [ ]:
os.chdir(ISCE_WORKSPACE)
cmd_reference = "bash ./run_files/run_02_reference"
subprocess.run(cmd_reference, shell=True, check=True)

In [ ]:
os.chdir(ISCE_WORKSPACE)
cmd_focus_split = "bash ./run_files/run_03_focus_split"
subprocess.run(cmd_focus_split, shell=True, check=True)

In [ ]:
os.chdir(ISCE_WORKSPACE)
cmd_coarseResamp = "bash ./run_files/run_04_geo2rdr_coarseResamp"
subprocess.run(cmd_coarseResamp, shell=True, check=True)

In [ ]:
os.chdir(ISCE_WORKSPACE)
cmd_refineSecondaryTiming = "bash ./run_files/run_05_refineSecondaryTiming"
subprocess.run(cmd_refineSecondaryTiming, shell=True, check=True)

In [ ]:
os.chdir(ISCE_WORKSPACE)
cmd_invertMisreg = "bash ./run_files/run_06_invertMisreg"
subprocess.run(cmd_invertMisreg, shell=True, check=True)

In [ ]:
os.chdir(ISCE_WORKSPACE)
cmd_fineResamp = "bash ./run_files/run_07_fineResamp"
subprocess.run(cmd_fineResamp, shell=True, check=True)

In [ ]:
os.chdir(ISCE_WORKSPACE)
cmd_grid_baseline = "bash ./run_files/run_08_grid_baseline"
subprocess.run(cmd_grid_baseline, shell=True, check=True)

In [ ]:
os.chdir(ISCE_WORKSPACE)
cmd_igram = "bash ./run_files/run_09_igram"
subprocess.run(cmd_igram, shell=True, check=True)